In [8]:
# %pip install pandas
# %pip install sqlalchemy
# %pip install os
import pandas as pd 
import os
from sqlalchemy import create_engine
import logging
import time

| Module                       | Purpose                              | Simple Meaning                 |
| ---------------------------- | ------------------------------------ | ------------------------------ |
| **pandas**                   | Reading + handling CSV data          | Works with data tables         |
| **os**                       | Interact with computer files/folders | See all files, build paths     |
| **sqlalchemy.create_engine** | Connect to databases                 | Save data into a database      |
| **logging**                  | Save program messages into a file    | Keep a record of what happened |
| **time**                     | Measure execution time               | Check how long something takes |


In [ ]:
logging.basicConfig(
    filename = "Logs/Ingestion_DB.log",
    level= logging.DEBUG,
    format= "%(asctime)s - %(levelname)s - %(message)s",
    filemode= "a"
)

engine = create_engine("sqlite:///Inventory.db")

### `logging.basicConfig()` 
- prepares Python’s logging system so your program can record important information in a clean and organized way.
---
#### ✔ 1. `filename = "G:/Kaif/Code/Logs/Ingestion_DB.log"`
- This tells Python to create a log file at this location.
--- 
#### ✔ 2. `level = logging.DEBUG`
- This sets the minimum level of messages you want to save.
- DEBUG is the lowest level → it saves everything:
    - info
    - warnings
    - errors
    - debug messages
--- 
#### ✔ 3. `format = "%(asctime)s - %(levelname)s - %(message)s"`
This controls how each log line looks.
Example log line:

`2025-03-12 14:32:45 - INFO - Ingesting Sales.csv in db`


- `%(asctime)s` → Time of the log
- `%(levelname)s` → Log type (INFO/ERROR/DEBUG)
- `%(message)s` → The actual message you write
---
#### ✔ 4.` filemode = "a"`
- "a" means append, i.e., add new logs to the existing file.
- It will not delete old logs
---
#### ✅ Explanation of Database Engine Setup
`engine = create_engine("sqlite:///Inventory.db")`

This line creates a connection to an SQLite database.

##### ✔ How does create_engine() work?
- It tells SQLAlchemy to connect to a database.

##### ✔ Why do we need engine?
Because Pandas needs this engine to push data into the database:

`df.to_sql(table_name, con=engine)`

The engine acts like a bridge between Pandas and the Database.

In [10]:
def ingest_db(file_path, table_name, engine, chunksize=50000):
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk.to_sql(table_name, con=engine, if_exists="append", index=False)


In [ ]:
def load_raw_data():
    start = time.time()

    data_path = "data"

    for file in os.listdir(data_path):
        if file.endswith(".csv"):
            file_path = os.path.join(data_path, file)
            logging.info(f"Ingesting {file} in db")

            # send file path, not whole df
            ingest_db(file_path, file[:-4], engine)

    end = time.time()
    total_time = (end-start)/60
    logging.info("--------------Ingestion Completed--------------")
    logging.info(f"Total Time Taken: {total_time} minutes")


if __name__ == "__main__":
    load_raw_data()


#### **1. Start a timer**

```start = time.time()```
- This records the current time.
- We will later calculate how long the full ingestion took.
---
#### **2. Define folder path**

`data_path = "G:/Kaif/Code/data"`
- This is the folder where all your CSV files are stored.
---
#### **3. Loop through every file in the folder**

`for file in os.listdir(data_path):`
- os.listdir() gives a list of all files in the folder.
---
#### **4. Create full file path**

`file_path = os.path.join(data_path, file)`
- `os.path.join()` safely joins folder + file name.
- Example: `"G:/Kaif/Code/data" + "sales.csv"`
→ `"G:/Kaif/Code/data/sales.csv"`
---
#### **5. Log which file is being ingested**
`logging.info(f"Ingesting {file} in db")`
- Writes a message inside your .log file.
---
#### **6. Call the ingestion function**
`ingest_db(file_path, file[:-4], engine)`

**Important:**
- file_path = the actual CSV file location.
- file[:-4] removes .csv
    Example: "sales.csv"[:-4] → "sales"
- engine connects to the database.
---
#### **7. End the timer**
`end = time.time()`
`total_time = (end - start) / 60`
- Calculates total time taken (converted to minutes).

In [13]:
# def load_raw_data():
#     start = time.time()

#     for file in os.listdir("G:/Kaif/Code/data"):
#         if ".csv" in file:
#             df = pd.read_csv("G:/Kaif/Code/data/"+file)
#             logging.info(f"Ingesting {file} in db")
#             ingest_db(df,file[:-4], engine)

#     end = time.time()
#     total_time = (end-start)/60
#     logging.info("--------------Ingestion Completed--------------")

#     logging.info(f"\nTotal Time Taken: {total_time} minutes")

# if __name__ == "__main__":
#     load_raw_data()